### setup

In [1]:
import requests
import json

token = "abcd1234
headers = {"Authorization": f"Token {token}"}

url_stub = "http://0.0.0.0:8000/api/v1" # or https://validation-server-stg.urban.org/api/v1/

### /api/v1/command/

#### post a new command

* `POST` upon new file upload

In [2]:
sanitized_command_input = {
        "epsilon": 1,
        "analysis_query": "SELECT mars, COUNT(recid) as n FROM puf.puf_kueyama GROUP BY mars",
        "transformation_query": "CREATE TABLE puf.puf_kueyama AS SELECT * FROM puf.puf"
    }

payload = {
    "researcher_id": 1,
    "command_type": 2,
    #"filename" : "command.sql", -- TODO
    "sanitized_command_input": json.dumps(sanitized_command_input)
}

r = requests.post(f"{url_stub}/command/", headers=headers, data=payload)

In [3]:
print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
researcher_id 1
command_id 4
command_type 2
sanitized_command_input {'epsilon': 1, 'analysis_query': 'SELECT mars, COUNT(recid) as n FROM puf.puf_kueyama GROUP BY mars', 'transformation_query': 'CREATE TABLE puf.puf_kueyama AS SELECT * FROM puf.puf'}


#### get a command

In [4]:
command_id = r.json()["command_id"]
r = requests.get(f"{url_stub}/command/{command_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
researcher_id 1
command_id 4
command_type 2
sanitized_command_input {'epsilon': 1, 'analysis_query': 'SELECT mars, COUNT(recid) as n FROM puf.puf_kueyama GROUP BY mars', 'transformation_query': 'CREATE TABLE puf.puf_kueyama AS SELECT * FROM puf.puf'}


#### rename a command

* The mockup allows you to rename a command

In [5]:
# TODO

### /api/v1/synthetic-data-run/

#### post a run

* Can this be created directly in Django when a command is `POST`-ed?
* TO-DO: Trigger the backend AWS Lambda function to run the SmartNoise query

In [6]:
payload = {
    "command_id": command_id, # from the POST to command
    "epsilon": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/synthetic-data-run/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
command_id 4
run_id 5
epsilon 1.00
date_time_run_submitted 2021-07-22T20:49:18+0000


#### get a synthetic data run

In [7]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/synthetic-data-run/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
run_id 5
epsilon 1.00
date_time_run_submitted 2021-07-22T20:49:18+0000


### /api/v1/synthetic-data-result/

#### post a synthetic result

* Will be done from the backend

In [8]:
result = {
    "status": "success",
    "data": "some_data",
    "accuracy": "some_accuracy"
}

payload = {
    "command_id": command_id,
    "run_id": run_id,
    "result": json.dumps(result),
    "privacy_budget_used": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/synthetic-data-result/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
command_id 4
run_id 5
result {'status': 'success', 'data': 'some_data', 'accuracy': 'some_accuracy'}
privacy_budget_used 1.00


#### get a synthetic data result

* What is the best way to signal if a run failed?

In [9]:
r = requests.get(f"{url_stub}/synthetic-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
run_id 5
result {'data': 'some_data', 'status': 'success', 'accuracy': 'some_accuracy'}
privacy_budget_used 1.00


### /api/v1/confidential-data-run

#### post a confidential data run

* Will actually be multiple `POST` requests

In [10]:
payload = {
    "command_id": command_id, # from the POST to command
    "epsilon": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/confidential-data-run/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
command_id 4
run_id 3
epsilon 1.00
date_time_run_submitted 2021-07-22T20:49:19+0000


#### get a confidential data run

In [11]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-run/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
run_id 3
epsilon 1.00
date_time_run_submitted 2021-07-22T20:49:19+0000


### /api/v1/confidential-data-result/

#### post a confidential data result

* Will be done from the backend

In [12]:
result = {
    "status": "success",
    "data": "some_data",
    "accuracy": "some_accuracy"
}

payload = {
    "command_id": command_id,
    "run_id": run_id,
    "result": json.dumps(result),
    "privacy_budget_used": 1.00,
    "display_results_decision": True,
    "release_results_decision": False
}

r = requests.post(f"{url_stub}/confidential-data-result/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
command_id 4
run_id 3
result {'status': 'success', 'data': 'some_data', 'accuracy': 'some_accuracy'}
display_results_decision True
release_results_decision False
privacy_budget_used 1.00


#### get a confidential data result

In [13]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
run_id 3
result {'data': 'some_data', 'status': 'success', 'accuracy': 'some_accuracy'}
display_results_decision True
release_results_decision False
privacy_budget_used 1.00


### /api/v1/public-user-budget/

#### get user budget

In [14]:
researcher_id = 1
r = requests.get(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
researcher_id 1
total_budget_allocated 100.00
total_budget_used 2.00


#### update user budget

In [15]:
payload = {'total_budget_allocated': '100.00', 'privacy_budget_used': '1'}

r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
researcher_id 1
total_budget_allocated 100.00
total_budget_used 3.00


#### don't go over budget

In [16]:
payload = {'total_budget_allocated': '100.00', 'privacy_budget_used': '1000'}

r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
print(r.text)

400
Bad Request
{"non_field_errors":["Cannot exceed budget allocation"]}
